<h1>Тестовое задание по SQL</h1>

In [1]:
import sqlite3
import random
import datetime

In [3]:
connection = sqlite3.connect("test.sqlite")

In [4]:
cursor = connection.cursor()

<h3> Часть 1. Создадим необходимые таблицы </h3>

In [46]:

#Таблица пилотов
create_pilots_query = """
CREATE TABLE pilots (
  pilot_id INTEGER PRIMARY KEY AUTOINCREMENT,
  name varchar(50),
  age INTEGER,
  rank INTEGER,
  education_level varchar(50)
);
"""

cursor.execute(create_pilots_query)
connection.commit()

In [47]:
#Заполним случайными данными
PILOTS_NUM = 10
for i in range(PILOTS_NUM):
    name = (''.join([chr(random.randint(97,122))for c in range(random.randint(3,7))])).capitalize()
    age = random.randint(18,80)
    rank = random.randint(1,6)
    education_level = random.sample(['poor','fancy'],1)[0]
    add_pilot_query = 'INSERT INTO pilots VALUES (?, ?, ?, ?, ?)'
    cursor.execute(add_pilot_query,(i+1,name,age,rank,education_level))
    connection.commit()

In [48]:
#Посмотрим, что получилось
select_query = 'SELECT * FROM pilots'
cursor.execute(select_query)
results = cursor.fetchall()
results

[(1, 'Dvwbeta', 68, 6, 'poor'),
 (2, 'Flu', 32, 4, 'poor'),
 (3, 'Bygalaa', 47, 6, 'fancy'),
 (4, 'Jou', 71, 4, 'fancy'),
 (5, 'Dqqjla', 80, 3, 'poor'),
 (6, 'Xuuv', 66, 4, 'fancy'),
 (7, 'Dtvu', 70, 2, 'poor'),
 (8, 'Qeor', 73, 2, 'fancy'),
 (9, 'Vam', 73, 5, 'fancy'),
 (10, 'Mpjoo', 49, 2, 'fancy')]

In [49]:
# Таблица самолетов
create_crafts_query = """
CREATE TABLE crafts (
  plane_id INTEGER PRIMARY KEY AUTOINCREMENT,
  capacity INTEGER,
  cargo_flg BOOL
);
"""

cursor.execute(create_crafts_query)
connection.commit()

In [50]:
#Заполним случайными данными
CRAFTS_NUM = 10
for i in range(CRAFTS_NUM):
    cap = random.randint(10,100)
    cflg = random.randint(0,1)
    add_craft_query = 'INSERT INTO crafts VALUES (?, ?, ?)'
    cursor.execute(add_craft_query,(i+1,cap,cflg))
    connection.commit()

In [51]:
#Посмотрим, что получилось
select_query = 'SELECT * FROM crafts'
cursor.execute(select_query)
results = cursor.fetchall()
results

[(1, 78, 0),
 (2, 43, 0),
 (3, 84, 0),
 (4, 34, 1),
 (5, 54, 0),
 (6, 74, 0),
 (7, 95, 0),
 (8, 23, 1),
 (9, 52, 0),
 (10, 56, 0)]

In [54]:
#Таблица рейсов
drop_flights_query = 'DROP TABLE flights'
cursor.execute(drop_flights_query)
connection.commit()
create_flights_query = """
CREATE TABLE flights (
flight_id INTEGER,
flight_dt DATE,
plane_id INTEGER,
first_pilot_id INTEGER,
second_pilot_id INTEGER,
destination varchar(10),
quantity INTEGER,
FOREIGN KEY (first_pilot_id) REFERENCES pilots(pilot_id),
FOREIGN KEY (second_pilot_id) REFERENCES pilots(pilot_id),
CONSTRAINT flight_id_dt PRIMARY KEY (flight_id, flight_dt)
);
"""

cursor.execute(create_flights_query)
connection.commit()

In [55]:
#Заполним случайными данными (первый и второй пилоты не должны совпадать)
import datetime
FLIGHTS_NUM = 10
for i in range(FLIGHTS_NUM):
    fid = random.randint(1,1000)
    fdt = datetime.date(2021, random.randint(1,12), random.randint(1,28))
    pid = random.randint(1,CRAFTS_NUM+1)
    fpid = random.randint(1,PILOTS_NUM+1)
    spid = fpid
    while spid == fpid: spid = random.randint(1,PILOTS_NUM+1)
    dest = random.sample(['SVO','VKO','DME','LED'],1)[0]
    quan = random.randint(1,100)
    add_fight_query = 'INSERT INTO flights VALUES (?, ?, ?, ?, ?, ?, ?)'
    cursor.execute(add_fight_query,(fid,fdt,pid,fpid,spid,dest,quan))
    connection.commit()

In [56]:
#Посмотрим, что получилось
select_query = 'SELECT * FROM flights'
cursor.execute(select_query)
results = cursor.fetchall()
results

[(75, '2021-02-26', 1, 1, 5, 'VKO', 33),
 (911, '2021-03-12', 10, 3, 10, 'VKO', 17),
 (854, '2021-06-18', 5, 10, 6, 'VKO', 2),
 (373, '2021-12-05', 9, 3, 8, 'SVO', 29),
 (329, '2021-12-18', 9, 4, 8, 'LED', 5),
 (536, '2021-06-17', 6, 10, 2, 'VKO', 80),
 (941, '2021-05-17', 6, 11, 5, 'VKO', 24),
 (373, '2021-08-28', 7, 6, 8, 'LED', 1),
 (545, '2021-03-01', 1, 6, 9, 'LED', 15),
 (831, '2021-04-24', 5, 10, 7, 'LED', 78)]

<h3>Задание 1.  Напишите запрос, который выведет пилотов, которые в качестве второго пилота в 
августе этого года ездили в аэропорт Шереметьево. </h3>

In [59]:
# В случайно сгенерированных данных такого пилота не оказалось, поэтому
# добавим нужный рейс. Второй пилот будет с id=1 и именем "Dvwbeta"
fid = random.randint(1,1000)
fdt = datetime.date(2021, 8, random.randint(1,28))
pid = random.randint(1,CRAFTS_NUM+1)
fpid = random.randint(1,PILOTS_NUM+1)
spid = 1
dest = 'SVO'
quan = random.randint(1,100)
add_fight_query = 'INSERT INTO flights VALUES (?, ?, ?, ?, ?, ?, ?)'
cursor.execute(add_fight_query,(fid,fdt,pid,fpid,spid,dest,quan))
connection.commit()

Собственно, запрос. Необходимо, чтобы вополнилось 2 условия: destination=SVO AND flight_dt LIKE '2021-08-_(полет в августе 2021). 
Выбираем всех вторых пилотов с этих рейсов и выводим их имена.

In [60]:
select_query = """SELECT name FROM pilots WHERE
pilot_id = (SELECT second_pilot_id FROM flights WHERE
destination = 'SVO' AND flight_dt LIKE '2021-08-__');"""
cursor.execute(select_query)
results = cursor.fetchall()
results

[('Dvwbeta',)]

<h3>Задание 2. Выведите пилотов старше 45 лет, которые совершили больше 30 полетов на грузовых 
самолетах. </h3>

Для начала добавим эти полеты в данные. Пусть это будет пилот с id=2 и именем Flu. Он может быть как первым, так и вторым пилотом, и самолет должен быть грузовым, т.е. plane id = 4 или 8

In [61]:
FLIGHTS_NUM = 31
for i in range(FLIGHTS_NUM):
    fid = random.randint(1,1000)
    fdt = datetime.date(2021, random.randint(1,12), random.randint(1,28))
    pid = random.sample([4,8],1)[0]
    if bool(random.getrandbits(1)):
        fpid = 2
        spid = fpid
        while spid == fpid: spid = random.randint(1,PILOTS_NUM+1)
    else:
        spid = 2
        fpid = spid
        while fpid == spid: fpid = random.randint(1,PILOTS_NUM+1) 
    dest = random.sample(['SVO','VKO','DME','LED'],1)[0]
    quan = random.randint(1,100)
    add_fight_query = 'INSERT INTO flights VALUES (?, ?, ?, ?, ?, ?, ?)'
    cursor.execute(add_fight_query,(fid,fdt,pid,fpid,spid,dest,quan))
    connection.commit()

Изменим его возраст на 46

In [62]:
make_him_46 = """UPDATE pilots
SET age =46
WHERE pilot_id = 2; """
cursor.execute(make_him_46)
connection.commit()
select_query = 'SELECT * FROM pilots'
cursor.execute(select_query)
results = cursor.fetchall()
results

[(1, 'Dvwbeta', 68, 6, 'poor'),
 (2, 'Flu', 46, 4, 'poor'),
 (3, 'Bygalaa', 47, 6, 'fancy'),
 (4, 'Jou', 71, 4, 'fancy'),
 (5, 'Dqqjla', 80, 3, 'poor'),
 (6, 'Xuuv', 66, 4, 'fancy'),
 (7, 'Dtvu', 70, 2, 'poor'),
 (8, 'Qeor', 73, 2, 'fancy'),
 (9, 'Vam', 73, 5, 'fancy'),
 (10, 'Mpjoo', 49, 2, 'fancy')]

Чтобы сформулировать запрос, будем думать с конца. Для начала, получим все рейсы на грузовых самолетах запросом 
SELECT *
FROM flights WHERE
plane_id IN(SELECT plane_id FROM CRAFTS WHERE cargo_flg = 1)
Нам нужны и первые и вторые пилоты, поэтому делаем 2 выборки и объединяем их командой UNION ALL во временную таблицу TMP, дальше используем GROUP BY по получившемуся столбцу и COUNT()>30, чтобы получить тех, кто совершил более 30 полетов.
Остается только совместить это условие с age>45 и вывести имена.

In [87]:
select_query ="""SELECT name FROM pilots WHERE
age>45 AND pilot_id IN 
(SELECT * FROM 
(SELECT  first_pilot_id
FROM flights WHERE
plane_id IN(SELECT plane_id FROM CRAFTS WHERE cargo_flg = 1)
UNION ALL
SELECT second_pilot_id
FROM flights WHERE
plane_id IN(SELECT plane_id FROM CRAFTS WHERE cargo_flg = 1)
) as TMP
GROUP BY TMP.first_pilot_id 
HAVING Count(first_pilot_id) > 30)"""
cursor.execute(select_query)
results = cursor.fetchall()
results

[('Flu',)]

<h3>Задание 3.  Выведите ТОП 10 пилотов-капитанов (first_pilot_id), которые перевезли наибольшее 
число пассажиров в этом году.  </h3>

Выберем полеты на пассажирских самолетах (plane_id IN(SELECT plane_id FROM CRAFTS WHERE cargo_flg = 0)) и полеты 2021 года (flight_dt LIKE '2021%'). 
Используем ORDER BY SUM(quantity) DESC, чтобы отсортировать по убыванию по общему кол-ву пассажиров.
Выведем имена 10ти пилотов с LIMIT 10

In [104]:
select_query = """SELECT name FROM pilots WHERE pilot_id IN 
(SELECT first_pilot_id
FROM flights WHERE
(plane_id IN(SELECT plane_id FROM CRAFTS WHERE cargo_flg = 0)) AND
(flight_dt LIKE '2021%')
GROUP BY first_pilot_id 
ORDER BY SUM(quantity)  DESC) LIMIT 10"""
cursor.execute(select_query)
results = cursor.fetchall()
results

[('Dvwbeta',), ('Flu',), ('Bygalaa',), ('Jou',), ('Xuuv',), ('Mpjoo',)]

В ответе всего 7 пилотов, т.к. их всего 10 в таблице и только 7 летали в качестве первого пилота пассажирского рейса